# Crop OMERO Image using Fiji


This notebook shows how to crop an OMERO image using the crop functionality available in [Fiji](https://imagej.net/Fiji). Fiji has been installed with few other plugins. For more information, check the Dockerfile.

First we add Fiji and the plugins to the classpath.

In [1]:
//Add dependencies to the classpath
%classpath add jar /opt/java-apps/Fiji.app/jars/*
%classpath add jar /opt/java-apps/Fiji.app/plugins/*
%classpath add jar /opt/java-apps/Fiji.app/plugins/OMERO.imagej-5.5.6/*

The following section shows:
 * how to connect to OMERO
 * how to open an OMERO image using Bio-Formats
 * how to crop an image using Fiji
 * how to save the generated image as OME-TIFF
 * how to import the created OME-TIFF to OMERO

In [5]:

g = new EasyForm("Enter credentials to connect to OMERO")
g.addTextField("Server").onInit({g['Server'] = "outreach.openmicroscopy.org"})
g.addTextField("UserName")
g.addPasswordField("Password")
g


In [4]:
ga = new EasyForm("Select the Image to analyze")
ga.addTextField("ImageID")
ga

In [7]:
//Java imports
import java.io.File
import java.lang.reflect.Array

import omero.gateway.Gateway
import omero.gateway.LoginCredentials
import omero.gateway.SecurityContext
import omero.gateway.facility.BrowseFacility
import omero.gateway.facility.DataManagerFacility
import omero.gateway.model.DatasetData
import omero.log.SimpleLogger

import ome.formats.importer.ImportConfig
import ome.formats.importer.OMEROWrapper
import ome.formats.importer.ImportLibrary
import ome.formats.importer.ImportCandidates
import ome.formats.importer.cli.ErrorHandler
import ome.formats.importer.cli.LoggingImportMonitor

import loci.formats.in.DefaultMetadataOptions
import loci.formats.in.MetadataLevel

import ij.IJ

//Connection information
HOST = g['Server']
USERNAME = g['UserName']
PASSWORD = g['Password']

PORT = 4064
group_id = "-1"

//Image to analyze
image_id = ga['ImageID'].toLong()

//Function to connect to OMERO
def connect_to_omero() {
    "Connect to OMERO"

    credentials = new LoginCredentials()
    credentials.getServer().setHostname(HOST)
    credentials.getServer().setPort(PORT)
    credentials.getUser().setUsername(USERNAME.trim())
    credentials.getUser().setPassword(PASSWORD.trim())
    simpleLogger = new SimpleLogger()
    gateway = new Gateway(simpleLogger)
    gateway.connect(credentials)
    return gateway

}

//Find the dataset matching the specified ID
def find_dataset(gateway, dataset_id) {
    "Load the Dataset"
    browse = gateway.getFacility(BrowseFacility)
    user = gateway.getLoggedInUser()
    ctx = new SecurityContext(user.getGroupId())
    return browse.findIObject(ctx, "omero.model.Dataset", dataset_id)
}

//Function to Open an OMERO image using Bio-Formats
def open_image_plus(HOST, USERNAME, PASSWORD, PORT, group_id, image_id) {
    "Open the image using the Bio-Formats Importer"

    StringBuilder options = new StringBuilder()
    options.append("location=[OMERO] open=[omero:server=")
    options.append(HOST)
    options.append("\nuser=")
    options.append(USERNAME.trim())
    options.append("\nport=")
    options.append(PORT)
    options.append("\npass=")
    options.append(PASSWORD.trim())
    options.append("\ngroupID=")
    options.append(group_id)
    options.append("\niid=")
    options.append(image_id)
    options.append("] ")
    options.append("windowless=true view=Hyperstack ")
    IJ.runPlugIn("loci.plugins.LociImporter", options.toString())
}

//Upload the generated image
def upload_image(paths, gateway, id) {
    "Upload an image to OMERO"

    user = gateway.getLoggedInUser()
    sessionKey = gateway.getSessionId(user)

    config = new ImportConfig()
    config.debug.set('false')
    config.hostname.set(HOST)
    config.sessionKey.set(sessionKey)
    dataset = find_dataset(gateway, id)

    store = config.createStore()
    reader = new OMEROWrapper(config)

    library = new ImportLibrary(store, reader)
    error_handler = new ErrorHandler(config)

    library.addObserver(new LoggingImportMonitor())
    candidates = new ImportCandidates(reader, paths, error_handler)
    containers = candidates.getContainers()
    containers.each() { c ->
        c.setTarget(dataset)
    }
    reader.setMetadataOptions(new DefaultMetadataOptions(MetadataLevel.ALL))
    return library.importCandidates(config, candidates)
}

// Connect to OMERO
println "connecting..."
gateway = connect_to_omero()

println "opening Image..."
// Open the Image using Bio-Formats
open_image_plus(HOST, USERNAME, PASSWORD, PORT, group_id, String.valueOf(image_id))

// Crop the image
println "cropping..."
IJ.makeRectangle(0, 0, 200, 200)
IJ.run("Crop")

// Save modified image as OME-TIFF using Bio-Formats Exporter
imp = IJ.getImage()
name = imp.getTitle().replaceAll("\\s","")
file = File.createTempFile(name, ".ome.tiff")
path_to_file = file.getAbsolutePath()
println  path_to_file
options = "outfile=" + path_to_file + " export compression=Uncompressed windowless=true"
IJ.runPlugIn("loci.plugins.LociExporter", options.toString())
imp.changes = false
imp.close()

// Create a Dataset
d = new DatasetData()
d.setName("Cropped Image")
dm = gateway.getFacility(DataManagerFacility)
user = gateway.getLoggedInUser()
ctx = new SecurityContext(user.getGroupId())
d = dm.createDataset(ctx, d, null)

// Import the generated OME-TIFF to OMERO
println "importing..."
str2d = new String[1]
str2d[0] = path_to_file
success = upload_image(str2d, gateway, d.getId())
println "imported"

// Close the connection
gateway.disconnect()
println "Done"
// delete the local OME-TIFF image
file.delete()


connecting...
opening Image...
cropping...
/tmp/ScreenShot2019-09-03at10.43.29.png2147109111564075421.ome.tiff
importing...
Image:55296


Other imported objects:
Fileset:39220


imported
Done


true